In [2]:
import pandas as pd, numpy as np, datetime as dt, re
import matplotlib.pyplot as plt, scipy.stats as stats
import sqlalchemy as sql

from db_info import connection_str

In [3]:
# reference: https://fansided.com/2015/12/21/nylon-calculus-101-possessions/#:~:text=How%20does%20stats.NBA.com,correctly%20factor%20in%20team%20rebounds.
def possessionCalculator(df):
    # equation used to properly account for rebounds as a factor of overall possessions.
    # original formula:
        # 0.5 * (FGA + (0.4*FTA) - 1.07*(ORB / (ORB  + Opp DRB)) * (FGA - FG) + TOV) + (Opp FGA + (0.4*Opp FTA) - 1.07*(Opp ORB / (Opp ORB + DRB)) * (Opp FGA - Opp FG) + Opp TOV)
    home_teams = df[lambda x: x.home_team_bin == 1].loc[:, ["fg", "fga", "fta", "orb", "drb", "tov", "game_code"]]
    away_teams = df[lambda x: x.home_team_bin == 0].loc[:, ["fg", "fga", "fta", "orb", "drb", "tov", "game_code"]]
    df_game_by_game = home_teams.merge(away_teams, on="game_code", suffixes=("_home", "_away")).sort_index()
    
    df_game_by_game["game_possessions"] = 0.5 * ((df_game_by_game["fga_home"] + 0.4*df_game_by_game["fta_home"] - (1.07* (df_game_by_game["orb_home"] / (df_game_by_game["orb_home"] + df_game_by_game["drb_away"])))\
                                                  * (df_game_by_game["fga_home"] - df_game_by_game["fg_home"]) + df_game_by_game["tov_home"] +\
                                                  (df_game_by_game["fga_away"] + 0.4*df_game_by_game["fta_away"] - (1.07* (df_game_by_game["orb_away"] / (df_game_by_game["orb_away"] + df_game_by_game["drb_home"])))\
                                                   * (df_game_by_game["fga_away"] - df_game_by_game["fg_away"]) + df_game_by_game["tov_away"]
                                                  )))
    
    return df.merge(df_game_by_game.loc[:, ["game_code", "game_possessions"]], on="game_code")

In [4]:
engine = sql.create_engine(connection_str)
conn = engine.connect()

In [5]:
df_odds = pd.read_sql("select * from odds", con=conn, index_col="id")
df_time = pd.read_sql("select * from time", con=conn, index_col="id", parse_dates="timestamp")
df_gamecodes = pd.read_sql("select * from gamecodes", con=conn, index_col="id", parse_dates="date")

In [6]:
df_odds.shape

(87282, 9)

In [7]:
df_time.shape

(764039, 5)

In [8]:
df_gamecodes.shape

(5133, 4)

In [9]:
df_time.head()

,timestamp,book,spread,total,game_id
id,,,,,
1,2021-10-30 08:10:27,BOVADA,-7 -110,+224.5 -110,1459741
2,2021-10-30 08:35:12,BOVADA,-7 -110,+223.5 -110,1459741
3,2021-10-30 15:43:56,BOVADA,-7 -110,+224 -110,1459741
4,2021-10-30 15:50:00,BOVADA,-7 -110,+224.5 -110,1459741
5,2021-10-30 16:06:15,BOVADA,-7 -110,+223.5 -110,1459741


In [10]:
df_gamecodes.head()

,home_abbv,away_abbv,date,game_id
id,,,,
1,UTA,SAC,2021-11-02 21:00:00,1459841
2,DAL,MIA,2021-11-02 19:30:00,1459836
3,CHR,CLE,2021-11-01 19:00:00,1459786
4,SAN,DAL,2021-11-03 20:30:00,1459896
5,ATL,WAS,2021-11-01 19:30:00,1459806


In [11]:
df_gamecodes["game_id"] = df_gamecodes.game_id.astype("str")
df_odds["game_id"] = df_odds.game_id.astype("str")
df_time["game_id"] = df_time.game_id.astype("str")

In [12]:
df_odds

,team_abbv,book,moneyline,spread,spread_odds,total,over_odds,under_odds,game_id
id,,,,,,,,,
1,SAC,Opening,305,10.0,-110,220.0,-110,-110,1459841
2,UTA,Opening,-455,-10.0,-110,220.0,-110,-110,1459841
3,SAC,BOVADA,310,9.0,-110,223.0,-110,-110,1459841
4,UTA,BOVADA,-415,-9.0,-110,223.0,-110,-110,1459841
5,SAC,BetOnline,340,9.0,-106,223.0,-110,-110,1459841
...,...,...,...,...,...,...,...,...,...
87278,PHO,MyBookie,475,10.0,-110,209.0,-110,-110,894160
87279,BOS,GTBets,-691,-10.0,-111,209.0,-107,-108,894160
87280,PHO,GTBets,492,10.0,-108,209.0,-107,-108,894160


In [118]:
def seasoner(date):
    if date <= dt.datetime(2018, 6, 9):
        return "17-18"
    elif date <= dt.datetime(2019, 6, 14):
        return "18-19"
    elif date <= dt.datetime(2020, 10, 12):
        return "19-20"
    elif date <= dt.datetime(2021, 7, 21):
        return "20-21"
    else:
        return "21-22"

In [14]:
df_gamecodes["season"] = df_gamecodes.date.apply(seasoner)

In [15]:
df_gamecodes.groupby(["season"]).count()

,home_abbv,away_abbv,date,game_id
season,,,,
17-18,1370,1370,1370,1370
18-19,1319,1319,1319,1319
19-20,1144,1144,1144,1144
20-21,1182,1182,1182,1182
21-22,118,118,118,118


In [16]:
conn.close()

In [17]:
engine_2 = sql.create_engine("sqlite:///../data/interim/NBA.db")
conn_2 = engine_2.connect()
df_player_basic= pd.read_sql("SELECT * FROM basic_stats WHERE name = 'Team Totals' AND timetype = 'TotalBasics'", con=conn_2)

In [ ]:
not_in_NBA = merged_df[merged_df.home_team.isnull()].sort_values(['game_code'])[lambda x: x.date < dt.datetime(2021, 10, 17)].game_code.values
not_in_SQL = merged_df[merged_df.home_abbv.isnull()].sort_values("game_code").game_code.unique()

In [ ]:
# feed game_codes into game.py
not_in_NBA

array(['202008150POR', '202012220BRK', '202012220LAL', '202012230BOS',
       '202012230CHI', '202012230CLE', '202012230DEN', '202012230IND',
       '202012230MEM', '202012230MIN', '202012230ORL', '202012230PHI',
       '202012230PHO', '202012230POR', '202105180BOS', '202105180IND',
       '202105190LAL', '202105190MEM', '202105200WAS', '202105210GSW'],
      dtype=object)

In [ ]:
# look up in odds shark and feed into shark.py
not_in_SQL

array(['201801290ATL', '201803180LAC', '201803180MIN', '201803180NOP',
       '201803180TOR', '201803220CHO', '201803220DAL', '201803220HOU',
       '201803220NOP', '201803220ORL', '201803220SAC', '201803250BRK',
       '201803250GSW', '201803250HOU', '201803250IND', '201803250MIL',
       '201803250OKC', '201803250SAC', '201803250TOR', '201803250WAS',
       '201803260CHO', '201803260DET', '201803260MIN', '201803260PHI',
       '201803260PHO', '201803310BOS', '201803310MIA', '201803310NYK',
       '201803310SAC', '201803310WAS', '201804050CLE', '201804050DEN',
       '201804050HOU', '201804050IND', '201804050MIL', '201804050UTA',
       '201804060BOS', '201804060DET', '201804060LAL', '201804060MEM',
       '201804060NYK', '201804060ORL', '201804060PHI', '201804060PHO',
       '201804060TOR', '201804060WAS', '201811190ATL', '201903260CHO',
       '201903260CLE', '201903260DAL', '201903260DEN', '201903260LAL',
       '201903260MIA', '201903260MIL', '201903260MIN', '201903260NOP',
      

In [ ]:
with open("../data/interim/bball_ref_additional.txt", "w") as file:
    for x in not_in_NBA:
        file.write(x + "\n")
    file.close()

In [18]:
df_player_basic.head(20)

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
0,14,Team Totals,240.0,38,92,0.413,12,30,0.400,19,...,6,6,14,17,107,0.0,0,TotalBasics,DAL,201810260TOR
1,110,Team Totals,240.0,44,91,0.484,11,27,0.407,17,...,10,5,12,22,116,0.0,0,TotalBasics,TOR,201810260TOR
2,196,Team Totals,240.0,40,99,0.404,14,36,0.389,7,...,4,6,14,27,101,0.0,0,TotalBasics,BOS,201810190TOR
3,294,Team Totals,240.0,43,92,0.467,13,35,0.371,14,...,3,3,12,19,113,0.0,0,TotalBasics,TOR,201810190TOR
4,392,Team Totals,240.0,34,85,0.400,7,19,0.368,29,...,4,0,16,25,104,0.0,0,TotalBasics,CLE,201810170TOR
5,490,Team Totals,240.0,45,92,0.489,14,33,0.424,12,...,6,7,9,29,116,0.0,0,TotalBasics,TOR,201810170TOR
6,588,Team Totals,240.0,41,97,0.423,9,28,0.321,15,...,5,5,12,14,106,0.0,0,TotalBasics,CHO,201810220TOR
7,686,Team Totals,240.0,50,87,0.575,15,39,0.385,12,...,10,8,13,21,127,0.0,0,TotalBasics,TOR,201810220TOR
8,784,Team Totals,240.0,40,96,0.417,15,30,0.500,10,...,10,5,15,17,105,0.0,0,TotalBasics,MIN,201810240TOR
9,882,Team Totals,240.0,46,89,0.517,11,32,0.344,9,...,9,8,16,18,112,0.0,0,TotalBasics,TOR,201810240TOR


In [19]:
check_reference = df_player_basic.loc[:, ["team", "game_code"]]

In [20]:
abbv_regex = re.compile("[A-Z]{3}")

In [21]:
check_reference["home_team"] = check_reference.game_code.apply(lambda x: abbv_regex.search(x)[0])

In [22]:
check_reference["home_team_bin"] = (check_reference["team"] == check_reference["home_team"]).astype("int32")

In [23]:
check_reference

,team,game_code,home_team,home_team_bin
0,DAL,201810260TOR,TOR,0
1,TOR,201810260TOR,TOR,1
2,BOS,201810190TOR,TOR,0
3,TOR,201810190TOR,TOR,1
4,CLE,201810170TOR,TOR,0
...,...,...,...,...
9911,MEM,202105190MEM,MEM,1
9912,MEM,202105210GSW,GSW,0
9913,GSW,202105210GSW,GSW,1
9914,MEM,202008150POR,POR,0


In [24]:
df_gamecodes.shape

(5133, 5)

In [25]:
df_gamecodes.home_abbv.unique()

array(['UTA', 'DAL', 'CHR', 'SAN', 'ATL', 'LAL', 'GS', 'IND', 'BKN',
       'MIN', 'SAC', 'CHI', 'WAS', 'CLE', 'LAC', 'BOS', 'MIL', 'PHI',
       'DET', 'ORL', 'PHO', 'MEM', 'MIA', 'NY', 'TOR', 'DEN', 'NOP',
       'POR', 'HOU', 'OKC'], dtype=object)

In [26]:
check_reference.team.unique()

array(['DAL', 'TOR', 'BOS', 'CLE', 'CHO', 'MIN', 'DET', 'NOP', 'NYK',
       'PHI', 'WAS', 'LAC', 'SAS', 'POR', 'BRK', 'IND', 'GSW', 'PHO',
       'OKC', 'ORL', 'CHI', 'UTA', 'HOU', 'ATL', 'MIL', 'DEN', 'SAC',
       'MIA', 'LAL', 'MEM'], dtype=object)

In [27]:
shark_teams_flipper = {'CHR': 'CHO', 'SAN': 'SAS', 'GS': 'GSW', 'BKN': 'BRK', 'NY': 'NYK'}
df_gamecodes.home_abbv = df_gamecodes.home_abbv.apply(lambda x: shark_teams_flipper[x] if x in shark_teams_flipper.keys() else x)
df_gamecodes.away_abbv = df_gamecodes.away_abbv.apply(lambda x: shark_teams_flipper[x] if x in shark_teams_flipper.keys() else x) 
# Transform game_id into bball ref code 
df_gamecodes["game_code"] = (df_gamecodes.date.apply(dt.datetime.strftime, format="%Y%m%d0")) + df_gamecodes.home_abbv

In [28]:
merged_df = df_gamecodes.merge(check_reference, "left", on="game_code")

In [29]:
merged_df[merged_df.season != "21-22"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10070 entries, 117 to 10187
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   home_abbv      10070 non-null  object        
 1   away_abbv      10070 non-null  object        
 2   date           10070 non-null  datetime64[ns]
 3   game_id        10070 non-null  object        
 4   season         10070 non-null  object        
 5   game_code      10070 non-null  object        
 6   team           10070 non-null  object        
 7   home_team      10070 non-null  object        
 8   home_team_bin  10070 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 786.7+ KB


In [30]:
completed_seasons = merged_df[merged_df.season != "21-22"]
completed_seasons[completed_seasons.team.isnull()]

,home_abbv,away_abbv,date,game_id,season,game_code,team,home_team,home_team_bin


In [31]:
check_reference[check_reference.home_team == "GSW"].sort_values("game_code")

,team,game_code,home_team,home_team_bin
8031,GSW,201710170GSW,GSW,1
8030,HOU,201710170GSW,GSW,0
8004,TOR,201710250GSW,GSW,0
8005,GSW,201710250GSW,GSW,1
8049,GSW,201710270GSW,GSW,1
...,...,...,...,...
9660,NOP,202105140GSW,GSW,0
9668,MEM,202105160GSW,GSW,0
9669,GSW,202105160GSW,GSW,1
9912,MEM,202105210GSW,GSW,0


In [28]:
bball_merged = check_reference.merge(df_gamecodes, "left", on="game_code")

In [29]:
bball_merged[bball_merged.home_abbv.isnull()]

,team,game_code,home_team,home_team_bin,home_abbv,away_abbv,date,game_id,season


In [30]:
# creating sample data

In [74]:
sample_cumulative = df_gamecodes.date.sort_values(ascending=True).apply(dt.datetime.strftime, format="%Y-%m-%d").drop_duplicates().reset_index(drop=True).sort_values()

In [75]:
winnings = pd.Series([np.random.randint(-25, 25) for x in range(len(sample_cumulative))], name="winnings")

In [96]:
sample_bar_data = pd.concat([sample_cumulative, winnings], axis=1)

In [110]:
sample_bar_data.date = pd.to_datetime(sample_bar_data.date)

In [111]:
full_range_index = pd.date_range(start_date, end_date, freq="D")

In [112]:
sample_bar_data.index = sample_bar_data.date
sample_bar_data = sample_bar_data.drop("date", axis=1)

In [138]:
# sample_bar_data.resample("1D").ffill().iloc[115:130, :]
# sample_bar_data.iloc[115:130, :]

In [141]:
line_chart_sample_df = sample_bar_data.reset_index()

In [147]:
book_summary_sample_df = pd.DataFrame({"Book": ['BetNow', 'BetOnline', 'Bovada', "GTBets", 'Intertops', 'MyBookie', 'Opening', 'Skybook', 'SportsBetting', 'Average', 'Best', 'Worst'], 
"Wins": [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 666, 237, 1800], "Losses": [99, 100, 102, 103, 104, 105, 106, 107, 108, 161, 223, 954], "Pushes": [2, 3, 4, 5, 6, 8, 9, 11, 21, 105, 523, 301],
"Win_Pct": [.542, .476, .505, .610, .714, .712, .505, .301, .494, .511, .35, .409],"Metric":["Spread", "Spread", "Spread", "Spread", "Spread", "Spread", "Spread", "Spread", "Spread", "Spread", "Spread", "Spread"],
"Season":["17-18", "17-18", "17-18", "17-18", "17-18", "17-18", "17-18", "17-18", "17-18", "17-18", "17-18", "17-18"]})

In [148]:
book_summary_sample_df

,Book,Wins,Losses,Pushes,Win_Pct,Metric,Season
0,BetNow,1000,99,2,0.542,Spread,17-18
1,BetOnline,1001,100,3,0.476,Spread,17-18
2,Bovada,1002,102,4,0.505,Spread,17-18
3,GTBets,1003,103,5,0.610,Spread,17-18
4,Intertops,1004,104,6,0.714,Spread,17-18
5,MyBookie,1005,105,8,0.712,Spread,17-18
6,Opening,1006,106,9,0.505,Spread,17-18
7,Skybook,1007,107,11,0.301,Spread,17-18
8,SportsBetting,1008,108,21,0.494,Spread,17-18
9,Average,666,161,105,0.511,Spread,17-18


In [160]:
df_team_toals_basic = df_player_basic.query("timetype == 'TotalBasics'")

In [32]:
df_advanced_totals = pd.read_sql("SELECT * FROM advanced_stats WHERE name = 'Team Totals'", con=conn_2)

In [168]:
team_rankings_sample = pd.DataFrame()

In [174]:
for i, team in enumerate(df_team_toals_basic.team.unique()):
    team_rankings_sample.loc[i, "Team"] = team
    team_rankings_sample.loc[i, "Avg_Spread"] = np.random.randint(1, 12) 
    team_rankings_sample.loc[i, "Scoring"] = np.random.randint(99, 114)
    team_rankings_sample.loc[i, "eFG"] = np.random.randint(38, 59) * .01
    team_rankings_sample.loc[i, "TOV"] = np.random.randint(10, 40) * .01
    team_rankings_sample.loc[i, "ORB"] = np.random.randint(12, 35) * .01
    team_rankings_sample.loc[i, "FT_per_FGA"] = np.random.randint(90, 272) * .001
    team_rankings_sample.loc[i, "Pace"] = np.random.randint(95, 120)
    team_rankings_sample.loc[i, "Ws"] = np.random.randint(33, 82)
    team_rankings_sample.loc[i, "Ls"] = np.random.randint(33, 82)
    team_rankings_sample.loc[i, "Ps"] = np.random.randint(1, 5)
    team_rankings_sample.loc[i, "Season"] = "17-18" 

In [175]:
team_rankings_sample

,Team,Avg_Spread,Scoring,eFG,TOV,ORB,FT_per_FGA,Pace,Ws,Ls,Ps,Season
0,DAL,6.0,110.0,0.50,0.22,0.25,0.123,110.0,49.0,39.0,3.0,17-18
1,TOR,8.0,103.0,0.51,0.12,0.21,0.147,117.0,67.0,57.0,1.0,17-18
2,BOS,1.0,99.0,0.50,0.13,0.23,0.187,104.0,62.0,66.0,2.0,17-18
3,CLE,9.0,107.0,0.50,0.24,0.26,0.174,109.0,72.0,66.0,3.0,17-18
4,CHO,8.0,102.0,0.45,0.16,0.14,0.115,116.0,56.0,58.0,3.0,17-18
5,MIN,7.0,103.0,0.47,0.21,0.22,0.212,110.0,58.0,34.0,1.0,17-18
6,DET,2.0,105.0,0.39,0.39,0.23,0.103,116.0,74.0,37.0,2.0,17-18
7,NOP,10.0,111.0,0.38,0.16,0.14,0.173,106.0,34.0,78.0,4.0,17-18
8,NYK,4.0,102.0,0.42,0.23,0.22,0.203,106.0,68.0,59.0,2.0,17-18
9,PHI,7.0,102.0,0.40,0.17,0.33,0.271,97.0,42.0,41.0,4.0,17-18


In [177]:
team_rankings_sample.to_sql()

,Team,Avg_Spread,Scoring,eFG,TOV,ORB,FT_per_FGA,Pace,Ws,Ls,Ps,Season
0,DAL,6.0,110.0,0.50,0.22,0.25,0.123,110.0,49.0,39.0,3.0,17-18
1,TOR,8.0,103.0,0.51,0.12,0.21,0.147,117.0,67.0,57.0,1.0,17-18
2,BOS,1.0,99.0,0.50,0.13,0.23,0.187,104.0,62.0,66.0,2.0,17-18
3,CLE,9.0,107.0,0.50,0.24,0.26,0.174,109.0,72.0,66.0,3.0,17-18
4,CHO,8.0,102.0,0.45,0.16,0.14,0.115,116.0,56.0,58.0,3.0,17-18
5,MIN,7.0,103.0,0.47,0.21,0.22,0.212,110.0,58.0,34.0,1.0,17-18
6,DET,2.0,105.0,0.39,0.39,0.23,0.103,116.0,74.0,37.0,2.0,17-18
7,NOP,10.0,111.0,0.38,0.16,0.14,0.173,106.0,34.0,78.0,4.0,17-18
8,NYK,4.0,102.0,0.42,0.23,0.22,0.203,106.0,68.0,59.0,2.0,17-18
9,PHI,7.0,102.0,0.40,0.17,0.33,0.271,97.0,42.0,41.0,4.0,17-18


In [212]:
four_factors = df_advanced_totals.loc[:, ["team", "efg_pct", "tov_pct", "orb_pct", "fta_per_fga_pct", "game_code"]]

In [213]:
df_player_basic

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
0,14,Team Totals,240.0,38,92,0.413,12,30,0.400,19,...,6,6,14,17,107,0.0,0,TotalBasics,DAL,201810260TOR
1,28,Team Totals,60.0,9,23,0.391,2,9,0.222,6,...,1,0,2,7,26,0.0,0,1Q,DAL,201810260TOR
2,42,Team Totals,60.0,13,23,0.565,4,7,0.571,4,...,2,3,3,3,34,0.0,0,2Q,DAL,201810260TOR
3,56,Team Totals,120.0,22,46,0.478,6,16,0.375,10,...,3,3,5,10,60,0.0,0,1H,DAL,201810260TOR
4,70,Team Totals,60.0,9,22,0.409,4,8,0.500,7,...,1,2,2,3,29,0.0,0,3Q,DAL,201810260TOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70029,965037,Team Totals,0.0,,,0.000,,,0.000,,...,,,,,,0.0,0,2Q,POR,202008150POR
70030,965050,Team Totals,0.0,,,0.000,,,0.000,,...,,,,,,0.0,0,1H,POR,202008150POR
70031,965063,Team Totals,0.0,,,0.000,,,0.000,,...,,,,,,0.0,0,3Q,POR,202008150POR
70032,965076,Team Totals,0.0,,,0.000,,,0.000,,...,,,,,,0.0,0,4Q,POR,202008150POR


In [218]:
df_player_basic = df_player_basic.query("timetype == 'TotalBasics'")

In [221]:
df_player_basic["home_team"] = df_player_basic.game_code.apply(lambda x: abbv_regex.search(x)[0])
df_player_basic["home_team_bin"] = (df_player_basic["team"] == df_player_basic["home_team"]).astype("int32")


ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [237]:
pts_and_possessions = possessionCalculator(df_player_basic).loc[:, ["team", "pts", "game_possessions", "game_code", "home_team_bin"]]

In [238]:
results_df = pts_and_possessions.merge(four_factors, on=["game_code", "team"])

In [239]:
results_df.orb_pct = results_df.orb_pct / 100
results_df.tov_pct = results_df.tov_pct / 100

In [240]:
results_df

,team,pts,game_possessions,game_code,home_team_bin,efg_pct,tov_pct,orb_pct,fta_per_fga_pct
0,DAL,107,100.722,201810260TOR,0,0.478,0.120,0.208,0.261
1,TOR,116,100.722,201810260TOR,1,0.544,0.107,0.261,0.220
2,BOS,101,97.7803,201810190TOR,0,0.475,0.119,0.302,0.101
3,TOR,113,97.7803,201810190TOR,1,0.538,0.107,0.267,0.207
4,CLE,104,100.326,201810170TOR,0,0.441,0.135,0.286,0.459
...,...,...,...,...,...,...,...,...,...
10471,MEM,100,97.4041,202105190MEM,1,0.483,0.082,0.216,0.267
10472,MEM,117,105.546,202105210GSW,0,0.485,0.106,0.302,0.265
10473,GSW,112,105.546,202105210GSW,1,0.511,0.170,0.262,0.213
10474,MEM,122,104.101,202008150POR,0,0.565,0.150,0.286,0.239


In [241]:
df_odds

,team_abbv,book,moneyline,spread,spread_odds,total,over_odds,under_odds,game_id
id,,,,,,,,,
1,SAC,Opening,305,10.0,-110,220.0,-110,-110,1459841
2,UTA,Opening,-455,-10.0,-110,220.0,-110,-110,1459841
3,SAC,BOVADA,310,9.0,-110,223.0,-110,-110,1459841
4,UTA,BOVADA,-415,-9.0,-110,223.0,-110,-110,1459841
5,SAC,BetOnline,340,9.0,-106,223.0,-110,-110,1459841
...,...,...,...,...,...,...,...,...,...
87278,PHO,MyBookie,475,10.0,-110,209.0,-110,-110,894160
87279,BOS,GTBets,-691,-10.0,-111,209.0,-107,-108,894160
87280,PHO,GTBets,492,10.0,-108,209.0,-107,-108,894160


In [271]:
book_info_df = df_odds.merge(df_gamecodes.loc[:, ["game_id", "game_code"]], on=["game_id"])[lambda x: x.spread != -999].reset_index(drop=True)

In [283]:
book_info_df.groupby(["game_code", "team_abbv"]).idxmin() # Would Be Highest Favored Line, Priceiest Moenyline,Lowest Total = min

moneyline  spread  spread_odds  total  over_odds  \
game_code    team_abbv                                                     
201710170CLE BOS            75680   75674        75674  75674      75672   
             CLE            75673   75673        75673  75675      75673   
201710170GSW GS             75779   75773        75773  75773      75773   
             HOU            75770   75770        75770  75772      75772   
201710180BOS BOS            75809   75801        75801  75807      75799   
...                           ...     ...          ...    ...        ...   
202111030SAC SAC              177     179          179    179        177   
202111030SAS DAL               50      50           52     48         52   
             SAN               63      49           49     49         53   
202111030WAS TOR              208     208          212    210        208   
             WAS              215     211          211    211        209   

                        under_odds  
game_code    team_abbv              
201710170CLE BOS             75672  
             CLE             75673  
201710170GSW GS              75771  
             HOU             75770  
201710180BOS BOS             75801  
...                            ...  
202111030SAC SAC               177  
202111030SAS DAL                48  
             SAN                49  
202111030WAS TOR               208  
             WAS               209  

[10112 rows x 6 columns]

In [ ]:
book_info_df.groupby("game_code").idxmax() # Would be Most Points Given to Underdog, Best Underdog Moneyline and Highest Total 

In [280]:
book_info_df.iloc[75772:75785, :]

,team_abbv,book,moneyline,spread,spread_odds,total,over_odds,under_odds,game_id,game_code
75772,HOU,BetOnline,375,9.5,-103,230.5,-115,-105,888635,201710170GSW
75773,GS,BetOnline,-450,-9.5,-117,230.5,-115,-105,888635,201710170GSW
75774,HOU,Intertops,355,9.5,-110,231.0,-110,-110,888635,201710170GSW
75775,GS,Intertops,-460,-9.5,-110,231.0,-110,-110,888635,201710170GSW
75776,HOU,SportsBetting,375,9.5,-103,230.5,-115,-105,888635,201710170GSW
75777,GS,SportsBetting,-450,-9.5,-117,230.5,-115,-105,888635,201710170GSW
75778,HOU,BetNow,395,9.5,-110,231.0,-110,-110,888635,201710170GSW
75779,GS,BetNow,-495,-9.5,-110,231.0,-110,-110,888635,201710170GSW
75780,HOU,MyBookie,370,9.5,-110,231.0,-110,-110,888635,201710170GSW
75781,GS,MyBookie,-495,-9.5,-110,231.0,-110,-110,888635,201710170GSW


In [285]:
results_df["Model_Prediction"] = 222
results_df["Best_Spread"] = -12
results_df["Best_Moneyline"] = -100
results_df["Best_Total"] = 223.5
results_df["Spread_Grade"] = "WIN"
results_df["Total_Grade"] = "PUSH"
results_df["Moneyline_Grade"] = "LOSS"

In [288]:
results_df["Best_Book"] = "BetOnline"

In [289]:
team_rankings_sample

,Team,Avg_Spread,Scoring,eFG,TOV,ORB,FT_per_FGA,Pace,Ws,Ls,Ps,Season
0,DAL,6.0,110.0,0.50,0.22,0.25,0.123,110.0,49.0,39.0,3.0,17-18
1,TOR,8.0,103.0,0.51,0.12,0.21,0.147,117.0,67.0,57.0,1.0,17-18
2,BOS,1.0,99.0,0.50,0.13,0.23,0.187,104.0,62.0,66.0,2.0,17-18
3,CLE,9.0,107.0,0.50,0.24,0.26,0.174,109.0,72.0,66.0,3.0,17-18
4,CHO,8.0,102.0,0.45,0.16,0.14,0.115,116.0,56.0,58.0,3.0,17-18
5,MIN,7.0,103.0,0.47,0.21,0.22,0.212,110.0,58.0,34.0,1.0,17-18
6,DET,2.0,105.0,0.39,0.39,0.23,0.103,116.0,74.0,37.0,2.0,17-18
7,NOP,10.0,111.0,0.38,0.16,0.14,0.173,106.0,34.0,78.0,4.0,17-18
8,NYK,4.0,102.0,0.42,0.23,0.22,0.203,106.0,68.0,59.0,2.0,17-18
9,PHI,7.0,102.0,0.40,0.17,0.33,0.271,97.0,42.0,41.0,4.0,17-18


In [291]:
book_summary_sample_df

,Book,Wins,Losses,Pushes,Win_Pct,Metric,Season
0,BetNow,1000,99,2,0.542,Spread,17-18
1,BetOnline,1001,100,3,0.476,Spread,17-18
2,Bovada,1002,102,4,0.505,Spread,17-18
3,GTBets,1003,103,5,0.610,Spread,17-18
4,Intertops,1004,104,6,0.714,Spread,17-18
5,MyBookie,1005,105,8,0.712,Spread,17-18
6,Opening,1006,106,9,0.505,Spread,17-18
7,Skybook,1007,107,11,0.301,Spread,17-18
8,SportsBetting,1008,108,21,0.494,Spread,17-18
9,Average,666,161,105,0.511,Spread,17-18


In [76]:
def date_fix(game_code):
    game_code = game_code[:-4]

    return dt.datetime(int(game_code[:4]), int(game_code[4:6]), int(game_code[6:]))

In [302]:
results_df["date"] = results_df["game_code"].apply(date_fix)

In [303]:
results_df

,team,pts,game_possessions,game_code,home_team_bin,efg_pct,tov_pct,orb_pct,fta_per_fga_pct,Model_Prediction,Best_Spread,Best_Moneyline,Best_Total,Spread_Grade,Total_Grade,Moneyline_Grade,Best_Book,date
0,DAL,107,100.722,201810260TOR,0,0.478,0.120,0.208,0.261,222,-12,-100,223.5,WIN,PUSH,LOSS,BetOnline,2018-10-26
1,TOR,116,100.722,201810260TOR,1,0.544,0.107,0.261,0.220,222,-12,-100,223.5,WIN,PUSH,LOSS,BetOnline,2018-10-26
2,BOS,101,97.7803,201810190TOR,0,0.475,0.119,0.302,0.101,222,-12,-100,223.5,WIN,PUSH,LOSS,BetOnline,2018-10-19
3,TOR,113,97.7803,201810190TOR,1,0.538,0.107,0.267,0.207,222,-12,-100,223.5,WIN,PUSH,LOSS,BetOnline,2018-10-19
4,CLE,104,100.326,201810170TOR,0,0.441,0.135,0.286,0.459,222,-12,-100,223.5,WIN,PUSH,LOSS,BetOnline,2018-10-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10471,MEM,100,97.4041,202105190MEM,1,0.483,0.082,0.216,0.267,222,-12,-100,223.5,WIN,PUSH,LOSS,BetOnline,2021-05-19
10472,MEM,117,105.546,202105210GSW,0,0.485,0.106,0.302,0.265,222,-12,-100,223.5,WIN,PUSH,LOSS,BetOnline,2021-05-21
10473,GSW,112,105.546,202105210GSW,1,0.511,0.170,0.262,0.213,222,-12,-100,223.5,WIN,PUSH,LOSS,BetOnline,2021-05-21
10474,MEM,122,104.101,202008150POR,0,0.565,0.150,0.286,0.239,222,-12,-100,223.5,WIN,PUSH,LOSS,BetOnline,2020-08-15


In [304]:
team_rankings_sample.to_csv("../data/interim/power_rankings_sample.csv", index=False)
book_summary_sample_df.to_csv("../data/interim/book_summary_sample.csv", index=False)
line_chart_sample_df.to_csv("../data/interim/line_chart_sample.csv", index=False)
results_df.to_csv("../data/interim/results_sample.csv", index=False)

In [4]:
# Sunday

In [5]:
engine = sql.create_engine(connection_str)
conn = engine.connect()
df_odds = pd.read_sql("select * from odds", con=conn, index_col="id")
df_time = pd.read_sql("select * from time", con=conn, index_col="id", parse_dates="timestamp")
df_gamecodes = pd.read_sql("select * from gamecodes", con=conn, index_col="id", parse_dates="date")

In [117]:
def seasoner(date):
    if date <= dt.datetime(2018, 6, 9):
        return "17-18"
    elif date <= dt.datetime(2019, 6, 14):
        return "18-19"
    elif date <= dt.datetime(2020, 10, 12):
        return "19-20"
    elif date <= dt.datetime(2021, 7, 21):
        return "20-21"
    else:
        return "21-22"

In [11]:
abbv_regex = re.compile("[A-Z]{3}")

In [7]:
df_gamecodes["season"] = df_gamecodes.date.apply(seasoner)

In [8]:
engine_2 = sql.create_engine("sqlite:///../data/interim/NBA.db")
conn_2 = engine_2.connect()
df_player_basic= pd.read_sql("SELECT * FROM basic_stats WHERE name = 'Team Totals' AND timetype = 'TotalBasics'", con=conn_2)
df_player_advanced = pd.read_sql("SELECT * FROM advanced_stats WHERE name = 'Team Totals'", con=conn_2)

In [12]:
df_player_basic["home_team"] = df_player_basic.game_code.apply(lambda x: abbv_regex.search(x)[0])
df_player_basic["home_team_bin"] = (df_player_basic["team"] == df_player_basic["home_team"]).astype("int32")

In [14]:
shark_teams_flipper = {'CHR': 'CHO', 'SAN': 'SAS', 'GS': 'GSW', 'BKN': 'BRK', 'NY': 'NYK'}
df_gamecodes.home_abbv = df_gamecodes.home_abbv.apply(lambda x: shark_teams_flipper[x] if x in shark_teams_flipper.keys() else x)
df_gamecodes.away_abbv = df_gamecodes.away_abbv.apply(lambda x: shark_teams_flipper[x] if x in shark_teams_flipper.keys() else x) 
# Transform game_id into bball ref code 
df_gamecodes["game_code"] = (df_gamecodes.date.apply(dt.datetime.strftime, format="%Y%m%d0")) + df_gamecodes.home_abbv

In [22]:
pts_and_possessions = possessionCalculator(df_player_basic).loc[:, ["team", "game_possessions", "game_code"]]

In [24]:
pts_and_possessions = possessionCalculator(df_player_basic).loc[:, ["team", "game_possessions", "game_code"]]
df_player_basic_w_pace = df_player_basic.merge(pts_and_possessions, on=["game_code", "team"])

In [27]:
game_results_df = df_player_basic_w_pace.merge(df_player_advanced, on=["game_code", "team"])

In [ ]:
df_odds
df_gamecodes
df_time

In [33]:
odds_w_codes = df_odds.merge(df_gamecodes.loc[:, ["game_id", "game_code", "date", "season"]], on="game_id")

In [38]:
df_time = df_time[(df_time.spread != "X") & (df_time.total != "X")]

In [42]:
game_results_df=game_results_df.loc[:, ['team','minutes_played_x', 'fg', 'fga', 'fg_pct', 'fg3',
'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast',
'stl', 'blk', 'tov', 'pf', 'pts', 'game_possessions',
'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct',
'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct',
'off_rtg','def_rtg', 'game_code', 'home_team_bin']]

In [46]:
game_results_df.loc[100:101, :]

,team,minutes_played_x,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,...,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg,game_code,home_team_bin
100,CHO,240.0,38,88,0.432,8,32,0.250,19,26,...,44.0,63.2,9.3,10.6,8.3,100.0,106.3,122.9,201812230BOS,0
101,BOS,240.0,43,86,0.500,13,39,0.333,20,25,...,56.0,69.8,8.3,7.1,12.6,100.0,122.9,106.3,201812230BOS,1


In [49]:
odds_w_codes[odds_w_codes.game_code == game_results_df.loc[100, "game_code"]]

,team_abbv,book,moneyline,spread,spread_odds,total,over_odds,under_odds,game_id,game_code,date,season
54830,CHR,Opening,220,7.0,-110,218.0,-110,-110,1028948,201812230BOS,2018-12-23 18:00:00,18-19
54831,BOS,Opening,-260,-7.0,-110,218.0,-110,-110,1028948,201812230BOS,2018-12-23 18:00:00,18-19
54832,CHR,BetOnline,250,7.5,-110,217.0,-110,-110,1028948,201812230BOS,2018-12-23 18:00:00,18-19
54833,BOS,BetOnline,-300,-7.5,-110,217.0,-110,-110,1028948,201812230BOS,2018-12-23 18:00:00,18-19
54834,CHR,Intertops,250,7.5,-115,217.0,-110,-110,1028948,201812230BOS,2018-12-23 18:00:00,18-19
54835,BOS,Intertops,-300,-7.5,-105,217.0,-110,-110,1028948,201812230BOS,2018-12-23 18:00:00,18-19
54836,CHR,SportsBetting,250,7.5,-110,217.0,-110,-110,1028948,201812230BOS,2018-12-23 18:00:00,18-19
54837,BOS,SportsBetting,-300,-7.5,-110,217.0,-110,-110,1028948,201812230BOS,2018-12-23 18:00:00,18-19
54838,CHR,BetNow,255,7.5,-110,217.0,-110,-110,1028948,201812230BOS,2018-12-23 18:00:00,18-19
54839,BOS,BetNow,-310,-7.5,-110,217.0,-110,-110,1028948,201812230BOS,2018-12-23 18:00:00,18-19


In [54]:
df_time[df_time.game_id == odds_w_codes[odds_w_codes.game_code == game_results_df.loc[100, "game_code"]].iloc[0, 8]]

,timestamp,book,spread,total,game_id
id,,,,,
392715,2018-12-22 15:04:10,BetNow,-6.5 -110,+222 -110,1028948
392725,2018-12-22 15:27:27,BetNow,-6.5 -110,+221 -110,1028948
392739,2018-12-22 15:27:28,BetNow,-7 -110,+221 -110,1028948
392749,2018-12-23 02:23:17,BetNow,-7.5 -110,+221 -110,1028948
392753,2018-12-23 05:26:10,BetNow,-7 -110,+221 -110,1028948
...,...,...,...,...,...
394384,2018-12-23 16:55:22,SportsBetting,-7.5 -110,+216 -110,1028948
394393,2018-12-23 17:33:38,SportsBetting,-7.5 -113,+216.5 -110,1028948
394407,2018-12-23 17:39:05,SportsBetting,-7.5 -114,+216.5 -110,1028948


In [56]:
game_results_df.loc[100:101, :].to_csv("../data/interim/games/results.csv", index=False)
odds_w_codes[odds_w_codes.game_code == game_results_df.loc[100, "game_code"]].to_csv("../data/interim/games/odds.csv", index=False)
df_time[df_time.game_id == odds_w_codes[odds_w_codes.game_code == game_results_df.loc[100, "game_code"]].iloc[0, 8]].to_csv("../data/interim/games/timeseries.csv", index=False)

In [38]:
df_player_basic

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
0,14,Team Totals,240.0,38,92,0.413,12,30,0.400,19,...,6,6,14,17,107,0.0,0,TotalBasics,DAL,201810260TOR
1,110,Team Totals,240.0,44,91,0.484,11,27,0.407,17,...,10,5,12,22,116,0.0,0,TotalBasics,TOR,201810260TOR
2,196,Team Totals,240.0,40,99,0.404,14,36,0.389,7,...,4,6,14,27,101,0.0,0,TotalBasics,BOS,201810190TOR
3,294,Team Totals,240.0,43,92,0.467,13,35,0.371,14,...,3,3,12,19,113,0.0,0,TotalBasics,TOR,201810190TOR
4,392,Team Totals,240.0,34,85,0.400,7,19,0.368,29,...,4,0,16,25,104,0.0,0,TotalBasics,CLE,201810170TOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9911,964586,Team Totals,240.0,40,90,0.444,7,22,0.318,13,...,4,11,9,19,100,0.0,0,TotalBasics,MEM,202105190MEM
9912,964668,Team Totals,265.0,40,98,0.408,15,35,0.429,22,...,14,6,13,23,117,0.0,0,TotalBasics,MEM,202105210GSW
9913,964795,Team Totals,265.0,42,94,0.447,12,34,0.353,16,...,7,7,21,23,112,0.0,0,TotalBasics,GSW,202105210GSW
9914,964914,Team Totals,240.0,45,92,0.489,14,25,0.560,18,...,5,6,18,25,122,0.0,0,TotalBasics,MEM,202008150POR


In [40]:
df_player_basic

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
0,14,Team Totals,240.0,38,92,0.413,12,30,0.400,19,...,6,6,14,17,107,0.0,0,TotalBasics,DAL,201810260TOR
1,110,Team Totals,240.0,44,91,0.484,11,27,0.407,17,...,10,5,12,22,116,0.0,0,TotalBasics,TOR,201810260TOR
2,196,Team Totals,240.0,40,99,0.404,14,36,0.389,7,...,4,6,14,27,101,0.0,0,TotalBasics,BOS,201810190TOR
3,294,Team Totals,240.0,43,92,0.467,13,35,0.371,14,...,3,3,12,19,113,0.0,0,TotalBasics,TOR,201810190TOR
4,392,Team Totals,240.0,34,85,0.400,7,19,0.368,29,...,4,0,16,25,104,0.0,0,TotalBasics,CLE,201810170TOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9911,964586,Team Totals,240.0,40,90,0.444,7,22,0.318,13,...,4,11,9,19,100,0.0,0,TotalBasics,MEM,202105190MEM
9912,964668,Team Totals,265.0,40,98,0.408,15,35,0.429,22,...,14,6,13,23,117,0.0,0,TotalBasics,MEM,202105210GSW
9913,964795,Team Totals,265.0,42,94,0.447,12,34,0.353,16,...,7,7,21,23,112,0.0,0,TotalBasics,GSW,202105210GSW
9914,964914,Team Totals,240.0,45,92,0.489,14,25,0.560,18,...,5,6,18,25,122,0.0,0,TotalBasics,MEM,202008150POR


In [41]:
df_player_basic["home_team"] = df_player_basic.game_code.apply(lambda x: abbv_regex.search(x)[0])
df_player_basic["home_team_bin"] = (df_player_basic["team"] == df_player_basic["home_team"]).astype("int32")

In [42]:
pts_and_possessions = possessionCalculator(df_player_basic).loc[:, ["team", "game_possessions", "game_code"]]
df_player_basic_w_pace = df_player_basic.merge(pts_and_possessions, on=["game_code", "team"])

In [52]:
pts_and_pace = df_player_basic_w_pace.loc[:, ["team", "game_code", "pts", "game_possessions","home_team_bin"]]

In [53]:
four_factors = df_advanced_totals.loc[:, ["team", "game_code", "efg_pct", "orb_pct", "tov_pct", "fta_per_fga_pct"]]

In [58]:
data_for_model = pts_and_pace.merge(four_factors, on=["game_code", "team"])

In [60]:
data_for_model.orb_pct = round((data_for_model.orb_pct / 100), 3)
data_for_model.tov_pct = round((data_for_model.tov_pct / 100), 3)

In [62]:
data_for_model.columns

Index(['team', 'game_code', 'pts', 'game_possessions', 'home_team_bin',
       'efg_pct', 'orb_pct', 'tov_pct', 'fta_per_fga_pct'],
      dtype='object')

In [63]:
# Rearrange Columns
data_for_model = data_for_model.loc[:, ['team', 'efg_pct', 'orb_pct', 'tov_pct', 'fta_per_fga_pct', 'pts',
'game_possessions','home_team_bin','game_code']]

In [71]:
# Find Repeat Columns
data_for_model.groupby("game_code").count().sort_values("team", ascending=False).reset_index().loc[:30, :]

,game_code,team,efg_pct,orb_pct,tov_pct,fta_per_fga_pct,pts,game_possessions,home_team_bin
0,201902010UTA,64,64,64,64,64,64,64,64
1,201812250UTA,64,64,64,64,64,64,64,64
2,201901250UTA,64,64,64,64,64,64,64,64
3,201901140UTA,64,64,64,64,64,64,64,64
4,201812270UTA,64,64,64,64,64,64,64,64
5,201812220UTA,64,64,64,64,64,64,64,64
6,201902090UTA,64,64,64,64,64,64,64,64
7,201902270UTA,64,64,64,64,64,64,64,64
8,201903020UTA,64,64,64,64,64,64,64,64
9,201901090UTA,64,64,64,64,64,64,64,64


In [73]:
data_for_model.drop_duplicates().groupby("game_code").count().sort_values("team", ascending=False)

,team,efg_pct,orb_pct,tov_pct,fta_per_fga_pct,pts,game_possessions,home_team_bin
game_code,,,,,,,,
201710170CLE,2,2,2,2,2,2,2,2
202001230BRK,2,2,2,2,2,2,2,2
202001240MIA,2,2,2,2,2,2,2,2
202001240GSW,2,2,2,2,2,2,2,2
202001240DET,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...
201812010NYK,2,2,2,2,2,2,2,2
201812010MIN,2,2,2,2,2,2,2,2
201812010HOU,2,2,2,2,2,2,2,2


In [74]:
data_for_model = data_for_model.drop_duplicates()

In [80]:
data_for_model["date"] = data_for_model.game_code.apply(date_fix)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
data_for_model["season"] = data_for_model.date.apply(seasoner)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [85]:
data_for_model.groupby("season").count()

,team,efg_pct,orb_pct,tov_pct,fta_per_fga_pct,pts,game_possessions,home_team_bin,game_code,date
season,,,,,,,,,,
17-18,2624,2624,2624,2624,2624,2624,2624,2624,2624,2624
18-19,2624,2624,2624,2624,2624,2624,2624,2624,2624,2624
19-20,2286,2286,2286,2286,2286,2286,2286,2286,2286,2286
20-21,2342,2342,2342,2342,2342,2342,2342,2342,2342,2342


In [90]:
df_gamecodes.drop_duplicates().groupby("season").count()

,home_abbv,away_abbv,date,game_id,game_code
season,,,,,
17-18,1311,1311,1311,1311,1311
18-19,1312,1312,1312,1312,1312
19-20,1143,1143,1143,1143,1143
20-21,1172,1172,1172,1172,1172
21-22,118,118,118,118,118


In [92]:
df_gamecodes = df_gamecodes.drop_duplicates()

In [91]:
data_for_model.groupby("season").count().team / 2

season
17-18    1312.0
18-19    1312.0
19-20    1143.0
20-21    1171.0
Name: team, dtype: float64

In [99]:
# Bring Team Stats to The Game Level; possessions, game_codes and season are game level information
away_teams = data_for_model.query("home_team_bin == 0").loc[:, ["team", "efg_pct", "orb_pct", "tov_pct", "fta_per_fga_pct", "pts", "game_code"]]
home_teams = data_for_model.query("home_team_bin == 1").loc[:, ["team", "efg_pct", "orb_pct", "tov_pct", "fta_per_fga_pct", "game_possessions","pts", "game_code", "date", "season"]]
game_by_game_df = pd.merge(home_teams, away_teams, on=["game_code"], suffixes=("_home", "_away"))


In [106]:
# Find Missing Games 

In [111]:
completed_seasons_gc = df_gamecodes.query("season != '21-22'")

In [115]:
gamecodes_and_stats = completed_seasons_gc.merge(game_by_game_df, "right", on=["game_code"])

In [116]:
gamecodes_and_stats[lambda x: x.season_x != x.season_y]

,home_abbv,away_abbv,date_x,game_id,season_x,game_code,team_home,efg_pct_home,orb_pct_home,tov_pct_home,...,game_possessions,pts_home,date_y,season_y,team_away,efg_pct_away,orb_pct_away,tov_pct_away,fta_per_fga_pct_away,pts_away
1296,GSW,TOR,2019-06-13 21:00:00,1124481,19-20,201906130GSW,GSW,0.556,0.282,0.147,...,94.588086,110,2019-06-13,18-19,TOR,0.555,0.262,0.112,0.354,114
1936,MIA,LAL,2020-10-11 19:30:00,1317281,20-21,202010110MIA,MIA,0.506,0.209,0.128,...,92.061205,93,2020-10-11,19-20,LAL,0.545,0.273,0.112,0.157,106
3600,CLE,GSW,2018-06-08 21:00:00,980519,18-19,201806080CLE,CLE,0.391,0.333,0.101,...,87.239054,85,2018-06-08,17-18,GSW,0.535,0.270,0.079,0.186,108


In [120]:
gamecodes_and_stats.loc[1, :]

home_abbv                               TOR
away_abbv                               BOS
date_x                  2018-10-19 20:00:00
game_id                             1026598
season_x                              18-19
game_code                      201810190TOR
team_home                               TOR
efg_pct_home                          0.538
orb_pct_home                          0.267
tov_pct_home                          0.107
fta_per_fga_pct_home                  0.207
game_possessions                    97.7803
pts_home                                113
date_y                  2018-10-19 00:00:00
season_y                              18-19
team_away                               BOS
efg_pct_away                          0.475
orb_pct_away                          0.302
tov_pct_away                          0.119
fta_per_fga_pct_away                  0.101
pts_away                                101
Name: 1, dtype: object

In [122]:
gamecodes_and_stats = gamecodes_and_stats.drop(["team_home", "team_away", "season_x", "date_y"], axis=1)

In [123]:
gamecodes_and_stats.columns

Index(['home_abbv', 'away_abbv', 'date_x', 'game_id', 'game_code',
       'efg_pct_home', 'orb_pct_home', 'tov_pct_home', 'fta_per_fga_pct_home',
       'game_possessions', 'pts_home', 'season_y', 'efg_pct_away',
       'orb_pct_away', 'tov_pct_away', 'fta_per_fga_pct_away', 'pts_away'],
      dtype='object')

In [126]:
gamecodes_and_stats = gamecodes_and_stats.loc[:, ['home_abbv', 'away_abbv','efg_pct_home', 'orb_pct_home', 'tov_pct_home', 'fta_per_fga_pct_home',
'efg_pct_away', 'orb_pct_away', 'tov_pct_away', 'fta_per_fga_pct_away', 'game_possessions', 'pts_home', 'pts_away', "date_x", 'season_y',
"game_id", "game_code"]]

In [127]:
# now results data with model variables has the results for all the games with both their shark game_id and bball ref game_code 
gamecodes_and_stats=gamecodes_and_stats.rename(columns={"season_y": "season", "date_x": "date"})

In [134]:
# Decouple the spread columns and total columns
df_time

,timestamp,book,spread,total,game_id
id,,,,,
1,2021-10-30 08:10:27,BOVADA,-7 -110,+224.5 -110,1459741
2,2021-10-30 08:35:12,BOVADA,-7 -110,+223.5 -110,1459741
3,2021-10-30 15:43:56,BOVADA,-7 -110,+224 -110,1459741
4,2021-10-30 15:50:00,BOVADA,-7 -110,+224.5 -110,1459741
5,2021-10-30 16:06:15,BOVADA,-7 -110,+223.5 -110,1459741
...,...,...,...,...,...
764035,2018-03-26 21:27:53,SportsBetting,+10 -107,+208 -110,894160
764036,2018-03-26 21:38:19,SportsBetting,+10 -105,+208.5 -110,894160
764037,2018-03-26 21:43:27,SportsBetting,+10 -105,+208.5 -115,894160


In [150]:
df_time = pd.read_sql("select * from time", con=conn, index_col="id", parse_dates="timestamp")

In [153]:
# Get Rid of Time Entries with Both Market Line Columns Null
time_df = df_time[~((df_time.spread == "X") & (df_time.total == "X"))]

In [154]:
time_df[time_df.total == "X"]

,timestamp,book,spread,total,game_id
id,,,,,
20,2021-10-30 09:52:43,Intertops,-7 -110,X,1459741
82,2021-10-29 15:56:43,GTBets,+3.5 -109,X,1459731
117,2021-10-30 00:30:57,Intertops,+4.5 -110,X,1459731
120,2021-10-30 08:18:24,Intertops,+3.5 -110,X,1459731
269,2021-10-30 09:52:43,Intertops,-2 -110,X,1459751
...,...,...,...,...,...
763458,2018-04-06 11:03:22,GTBets,+4 -113,X,894575
763578,2021-01-31 00:11:12,Intertops,+8 -110,X,1348586
763580,2021-01-31 08:07:05,Intertops,+8.5 -110,X,1348586


In [168]:
time_df.spread = time_df.spread.apply(lambda x: None if x == "X" else x)
time_df.total = time_df.total.apply(lambda x: None if x == "X" else x)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [192]:
def split_fix(combined_line, index):
    if combined_line != None:
        line_list = combined_line.split()
        if len(line_list) == 1 and index == 1:
            return -110.0
        line = line_list[index]
        if line == "Ev":
            line = 0.0
        
        return round(float(line), 2)

In [193]:
time_df["spread_line"] = time_df.spread.apply(split_fix, index=0)
time_df["spread_odds"] = time_df.spread.apply(split_fix, index=1)
time_df["total_line"] = time_df.total.apply(split_fix, index=0)
time_df["over_odds"] = time_df.total.apply(split_fix, index=1)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [195]:
time_df.sort_values("spread")

,timestamp,book,spread,total,game_id,spread_line,spread_odds,total_line,over_odds
id,,,,,,,,,
653179,2018-01-14 19:09:50,GTBets,+0.5 +100,None,891805,0.5,100.0,NaN,NaN
653206,2018-01-14 19:15:00,GTBets,+0.5 +100,None,891805,0.5,100.0,NaN,NaN
653259,2018-01-14 19:35:41,GTBets,+0.5 +100,None,891805,0.5,100.0,NaN,NaN
653283,2018-01-14 19:40:50,GTBets,+0.5 +100,None,891805,0.5,100.0,NaN,NaN
653331,2018-01-14 20:01:30,GTBets,+0.5 +100,None,891805,0.5,100.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
762579,2019-03-26 10:03:29,Intertops,None,+218 -110,1032028,NaN,NaN,218.0,-110.0
762673,2019-03-26 10:03:03,MyBookie,None,+218 -110,1032028,NaN,NaN,218.0,-110.0
763091,2021-01-31 09:31:48,Intertops,None,+218.5 -110,1350191,NaN,NaN,218.5,-110.0


In [197]:
time_df = time_df.reset_index(drop=True).drop(["spread", "total"], axis=1)

In [199]:
time_df.columns

Index(['timestamp', 'book', 'game_id', 'spread_line', 'spread_odds',
       'total_line', 'over_odds'],
      dtype='object')

In [202]:
time_df = time_df.loc[:, ['book', 'spread_line', 'spread_odds', 'total_line', 'over_odds', 'game_id', 'timestamp']]

In [206]:
# Get Rid of All Markets With Null Placeholder Value in Market Columns 
df_odds.sort_values("spread").head()

,team_abbv,book,moneyline,spread,spread_odds,total,over_odds,under_odds,game_id
id,,,,,,,,,
40764,DEN,SkyBook,-999,-999.0,-999,-999.0,-999,-999,1108450
43919,MIN,BetNow,-110,-999.0,-110,217.5,-110,-110,1031948
43910,CHI,SkyBook,-999,-999.0,-999,-999.0,-999,-999,1031943
43909,UTA,SkyBook,-999,-999.0,-999,-999.0,-999,-999,1031943
84256,WAS,SkyBook,-999,-999.0,-999,-999.0,-999,-999,889155


In [231]:
market_cols = list(df_odds.columns)
market_cols.pop(0)
market_cols.pop(0)
market_cols.pop(6)

'game_id'

In [232]:
df_odds_copy = df_odds.copy(deep=True)

In [233]:
for col in market_cols:
    df_odds[col] = df_odds[col].apply(lambda x: np.nan if x == -999.0 else x)

In [251]:
wanted_entries = df_odds.loc[:, market_cols].dropna(0, "all")

In [257]:
odds_summary_df = df_odds.loc[:, ["team_abbv", "book", "game_id"]].merge(wanted_entries, right_index=True, left_index=True)

In [260]:
odds_summary_df.groupby("game_id").count().sort_values("book")

,team_abbv,book,moneyline,spread,spread_odds,total,over_odds,under_odds
game_id,,,,,,,,
1302246,8,8,8,8,8,8,8,8
1261955,10,10,10,10,10,10,10,10
1261865,10,10,10,10,10,10,10,10
1261870,10,10,10,10,10,10,10,10
1261875,10,10,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...
894600,48,48,48,48,48,48,48,48
894595,48,48,48,48,48,48,48,48
894590,48,48,48,48,48,48,48,48


In [265]:
odds_summary_df = odds_summary_df.drop_duplicates()

In [272]:
# Now With The Cleaned Paired Down Game Results Data and The Cleaned Market Data (Odds and Time). I Now Have the Ability to Match Market Data to The Equivalent Game
ids = gamecodes_and_stats.loc[:, ["game_id", "game_code"]]

In [274]:
engine_2 = sql.create_engine("sqlite:///../data/interim/market.db")
conn_2 = engine_2.connect()

In [276]:
ids.to_sql("id", con=conn_2, index_label="id")

In [277]:
gamecodes_and_stats.to_sql("results", con=conn_2)

In [280]:
odds_summary_df.to_sql("market_summary", con=conn_2)

In [281]:
time_df.to_sql("timeseries", con=conn_2)

In [282]:
conn_2.close()